In [3]:
from torchvision.datasets import VOCDetection

In [17]:
train_dataset = VOCDetection(root="./data",
                             year="2007",
                             download=True,
                             image_set="trainval",
                             transform=None)
test_dataset = VOCDetection(root="./data",
                             year="2007",
                             download=True,
                             image_set="test",
                             transform=None)           

Using downloaded and verified file: ./data/VOCtrainval_06-Nov-2007.tar
Extracting ./data/VOCtrainval_06-Nov-2007.tar to ./data
Using downloaded and verified file: ./data/VOCtest_06-Nov-2007.tar
Extracting ./data/VOCtest_06-Nov-2007.tar to ./data


In [18]:
img, target = train_dataset[1]

In [20]:
target

{'annotation': {'folder': 'VOC2007',
  'filename': '000007.jpg',
  'source': {'database': 'The VOC2007 Database',
   'annotation': 'PASCAL VOC2007',
   'image': 'flickr',
   'flickrid': '194179466'},
  'owner': {'flickrid': 'monsieurrompu', 'name': 'Thom Zemanek'},
  'size': {'width': '500', 'height': '333', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'car',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '141', 'ymin': '50', 'xmax': '500', 'ymax': '330'}}]}}

In [23]:
len(test_dataset)

4952

In [85]:
!pip install selectivesearch
import selectivesearch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.4/315.4 kB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 4.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for selectivesearch: filename=selectivesearch-0.4-py3-none-any.whl size=4334 sha256=27048e2df9829c094845b5357f68a8ae45e112ed2ad33874483eff26793a7be4
  Stored in directory: /Users/h383kim/Library/Caches/pip/wheels/66/4e/88/6de23ce74be839a953498c4ebdfa809ad7da9422ac89ae856c
Successfully built selectivesearch


In [86]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
import torch.optim as optim
import torchvision
from torchvision import models
import matplotlib.pyplot as plt
#import selective_search

import cv2

import xml.etree.ElementTree as ET
from PIL import Image
import os

In [87]:
import sys

def ss_config(ss, img, mode):
    ss.setBaseImage(img)

    if mode == 's':
        ss.switchToSingleStrategy()
    elif mode == 'f':
        ss.switchToSelectiveSearchFast()
    elif mode == 'q':
        ss.switchToSelectiveSearchQuality()
    else:
        print("Re-enter the mode. s or f or q")
        sys.exit(1)


def selective_search(img, mode='q'):
    # Initiate Selective Search
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    # Configure the mode and image
    ss_config(ss, img, mode)

    # Process Selective-Search
    bboxes = ss.process() # bboxes: listof [x, y, w, h]
    bboxes[:, 2] += bboxes[:, 0] # bboxes -> listof [x, y, x + w, h]
    bboxes[:, 3] += bboxes[:, 1] # bboxes -> listof [x, y, x + w, h + y]

    return bboxes

In [88]:
def parse_xml_boxes(root):
    bboxes=[]
    for obj in root.findall('object'):
        cls_name = obj.find('name').text
        cls_idx = VOC_CLASSES.index(cls_name)
        xmin = int(obj.find('bndbox').find('xmin').text)
        ymin = int(obj.find('bndbox').find('ymin').text)
        xmax = int(obj.find('bndbox').find('xmax').text)
        ymax = int(obj.find('bndbox').find('ymax').text)
        bboxes.append([xmin, ymin, ymin, ymax, cls_idx]) 
    return bboxes

In [89]:
def compute_iou(boxA, boxB):
    # Compute the Intersection over Union (IoU) between two bounding boxes
    xA = max(boxA[0], boxB[0])  # max of xmin
    yA = max(boxA[1], boxB[1])  # max of ymin
    xB = min(boxA[2], boxB[2])  # min of xmax
    yB = min(boxA[3], boxB[3])  # min of ymax

    # Compute the area of the intersection rectangle
    interWidth = max(0, xB - xA + 1)
    interHeight = max(0, yB - yA + 1)
    interArea = interWidth * interHeight

    # Compute the area of both bounding boxes
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # Compute the Intersection over Union by dividing the intersection area by the sum of both areas minus the intersection
    iou = interArea / float(boxAArea + boxBArea - interArea) if interArea > 0 else 0.0

    return iou

In [90]:
VOC_CLASSES = [
    'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 
    'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

In [91]:
class CustomDataset(Dataset):
    def __init__(self, root, img_set='trainval', transform=None):
        self.root = root
        self.transform = transform
        self.img_set = img_set
        
        self.annotation_path = os.path.join(self.root, f'PASCAL_VOC_{self.img_set}', 'VOCdevkit', 'VOC2007', 'Annotations')
        self.img_path = os.path.join(self.root, f'PASCAL_VOC_{self.img_set}', 'VOCdevkit', 'VOC2007', 'JPEGimages')

        self.annotations = [os.path.join(self.annotation_path, xml) for xml in sorted(os.listdir(self.annotation_path)) if not xml.startswith('.')]
        self.images = [os.path.join(self.img_path, xml) for xml in sorted(os.listdir(self.img_path)) if not xml.startswith('.')]
        self.annotations = self.annotations[:10]
        self.images = self.images[:10]
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Load the image using Pillow
        image = Image.open(self.images[idx])
        image = np.array(image)
        # Load annotation
        tree = ET.parse(self.annotations[idx])
        gt_bboxes_labels = parse_xml_boxes(tree.getroot())
        gt_bboxes_labels = np.array(gt_bboxes_labels)
        bboxes, labels= gt_bboxes_labels[:, :4], gt_bboxes_labels[:, -1]
        # Apply transformation if any
        if self.transform:
            image = self.transform(image)
        # Convert data to tensors
        bboxes = torch.tensor(bboxes).float()
        labels = torch.tensor(labels).long()
        
        return image, bboxes, labels

In [92]:
from torchvision.ops import roi_pool

vgg = models.vgg16_bn(weights="IMAGENET1K_V1", progress=True)

class FastRCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Load pre-trained VGG16
        # vgg = models.vgg16(weights="IMAGENET1K_V1")
        # Convolutional layers
        self.features = vgg.features
        # ROI Pooling
        self.roi = roi_pool
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            # nn.Linear(4096, num_classes),
        )
        self.softmax = nn.Linear(4096, num_classes + 1)
        self.bbox = nn.Linear(4096, num_classes * 4)
        
    def forward(self, images, ROIs):
        # Compute feature maps
        feature_maps = self.features(images)
        #print(feature_maps.shape)
        # Apply ROI pooling
        pooled_features = self.roi(feature_maps, ROIs, output_size=(7, 7), spatial_scale=1.0/16.0)
        #print(pooled_features.shape)
        # Flatten pooled features
        x = pooled_features.view(pooled_features.size(0), -1) # .size(0) = Batch Size(N=64)
        # x: [64, 512, 7, 7]
        # Classifier
        x = self.classifier(x) # x: [64, 4096]
        
        # Outputs
        cls_score = self.softmax(x) # (64, num_classes + 1)
        bbox_pred = self.bbox(x) # (64, num_classes * 4)
        
        return cls_score, bbox_pred

In [93]:
import torch.nn.functional as F

class MultiTaskLoss(nn.Module):
    def __init__(self, lambda_reg=1):
        super().__init__()
        self.lambda_reg = lambda_reg
    
    def forward(self, cls_preds, bbox_preds, labels, bbox_targets, pos_indices, num_classes=20):
        # Cross entropy loss for classification
        classification_loss = F.cross_entropy(cls_preds, labels)
        # Bounding box loss
        '''
        bbox_preds: [64, 20, 4]
        bbox_targets: [len(pos_indices), 4]
        '''
        if len(pos_indices) > 0:
            pos_labels = labels[labels < 20] # Non-background
            bbox_preds = bbox_preds.view(-1, num_classes, 4) # Reshaping bbox_preds from [64, num_classes*4] to [64, num_classes, 4]
            bbox_preds_pos = bbox_preds[pos_indices, pos_labels, :]
            regression_loss = F.smooth_l1_loss(bbox_preds_pos, bbox_targets)
        else:
            regression_loss = torch.tensor(0.0, requires_grad=True).to(bbox_preds.device)
        
        totalLoss = classification_loss + regression_loss
        return totalLoss

In [94]:
from torch.utils.data import DataLoader

root_dir = '/Users/h383kim/pytorch/data'
# Create dataset and dataloader
train_dataset = CustomDataset(root_dir, img_set="trainval", transform=None)
test_dataset = CustomDataset(root_dir, img_set="test", transform=None)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [95]:
train_dataset[0]

(array([[[ 10,  10,  10],
         [ 13,  13,  13],
         [ 10,  10,  10],
         ...,
         [163, 188, 195],
         [162, 187, 194],
         [161, 186, 193]],
 
        [[  3,   4,   6],
         [  7,   9,   8],
         [ 11,  11,  13],
         ...,
         [163, 188, 195],
         [163, 188, 195],
         [163, 188, 195]],
 
        [[ 45,  49,  52],
         [ 24,  28,  29],
         [ 15,  16,  20],
         ...,
         [165, 190, 197],
         [165, 190, 197],
         [165, 190, 197]],
 
        ...,
 
        [[ 31,  12,   8],
         [ 36,  17,  11],
         [ 29,  10,   3],
         ...,
         [ 58,  72, 101],
         [ 61,  72, 104],
         [ 58,  71, 103]],
 
        [[ 36,   9,   2],
         [ 34,   7,   0],
         [ 38,   8,   0],
         ...,
         [ 66,  80, 109],
         [ 70,  83, 115],
         [ 70,  83, 115]],
 
        [[ 49,  17,   6],
         [ 60,  22,  11],
         [ 73,  25,  13],
         ...,
         [ 64,  79, 108],
  

In [96]:
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
fastrcnn = FastRCNN(20).to(DEVICE)

In [97]:
DEVICE

device(type='mps')

In [98]:
def bbox_transform(proposal, gt_box):
    g_xmin, g_ymin, g_w, g_h = gt_box[0], gt_box[1], gt_box[2] - gt_box[0], gt_box[3] - gt_box[1]
    p_xmin, p_ymin, p_w, p_h = proposal[0], proposal[1], proposal[2] - proposal[0], proposal[3] - proposal[1]
    # G_x, G_y, P_x, P_y = Center coordinates of gt_box and proposal
    G_x, G_y = g_xmin + 0.5*g_w, g_ymin + 0.5*g_h
    P_x, P_y = p_xmin + 0.5*p_w, p_ymin + 0.5*p_h
    # target delats(transformations) to be learned
    t_x = (G_x - P_x) / p_w
    t_y = (G_y - P_y) / p_h
    t_w = torch.log(g_w / p_w)
    t_h = torch.log(g_h / p_h)

    return torch.tensor([t_x, t_y, t_w, t_h])

In [99]:
def ss(img):
    # Move the tensor to CPU and convert to NumPy array
    img = img.cpu().numpy()
    
    labels, regions = selectivesearch.selective_search(img, scale=150, sigma=0.8, min_size=20)
    bboxes = [region['rect'] for region in regions]
    bboxes = np.array(bboxes)
    bboxes[:, 2] += bboxes[:, 0]
    bboxes[:, 3] += bboxes[:, 1]
    return bboxes

In [100]:
import random

def sample_proposals(proposals, gt_boxes, gt_labels):
    threshold_up = 0.5
    threshold_low = 0.1
    POS_NUM = 16
    NEG_NUM = 48
    positive_proposals, positive_labels, negative_proposals = [], [], []
    bbox_targets = [] # the target transformation from proposal to gt_bbox
    
    for proposal in proposals:
        best_iou = 0
        best_label = None
        best_gt_box = None
        
        for gt_box, gt_label in zip(gt_boxes, gt_labels):
            iou = compute_iou(proposal, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_label = gt_label
                best_gt_box = gt_box
        # If the proposal has IOU > 0.5
        if best_iou > threshold_up:
            positive_proposals.append(proposal)
            positive_labels.append(best_label)
            # Compute regression targets
            bbox_target = bbox_transform(proposal, best_gt_box)
            bbox_targets.append(bbox_target)
#        # Else if the proposal has 0.1 < IOU <= 0.5
#        elif best_iou > threshold_low:
#            negative_proposals.append(proposal)
#            # Doesn't need to keep track of label, because background label is fixed
#        else:
#            pass
        else:
            negative_proposals.append(proposal)
            
    # Choose ~16 positives and 48 negatives (More negatives if postives is less than 16)
    if len(positive_proposals) < POS_NUM:
        POS_NUM = len(positive_proposals)
        NEG_NUM = (64 - POS_NUM)
    
    pos_indices = random.sample(range(len(positive_proposals)), POS_NUM)
    neg_indices = random.sample(range(len(negative_proposals)), NEG_NUM)
    # Sampling takes place (Replacing the object and all its references if any)
    positive_proposals[:] = [positive_proposals[i] for i in pos_indices]
    positive_labels[:] = [positive_labels[i] for i in pos_indices]
    bbox_targets[:] = [bbox_targets[i] for i in pos_indices]
    negative_proposals[:] = [negative_proposals[i] for i in neg_indices]
    
    total_proposals = positive_proposals + negative_proposals
    total_labels = positive_labels + [20] * NEG_NUM # Background class label as 20
    total_labels = torch.tensor(total_labels, dtype=torch.int64)
    if len(bbox_targets) > 0:
        bbox_targets = torch.stack(bbox_targets)
    else:
        bbox_targets = torch.tensor(bbox_targets)
    return total_proposals, total_labels, bbox_targets, pos_indices
    

In [101]:
def train(model, dataloader, optimizer, loss_fn):
    model.train()
    train_loss, correct, train_acc = 0, 0, 0
    total_proposals = 0
    
    #for epoch in range(num_epochs):
    for image, boxes, labels in dataloader:
        # OpenCV version selective search
        np_img = np.squeeze(np.array(image))
        ROIs = selective_search(np_img)
        image = image.float().to(DEVICE)
        boxes = boxes.to(DEVICE)
        labels = labels.to(DEVICE)

        # Generate ROIs and choose 64 proposals(16 positive + 48 negative)
        # # pip install selectivesearch version
        # squeezed_img = torch.squeeze(image)
        # ROIs = ss(squeezed_img)
        '''
        ROIs: total proposals (positive + negative)
        ROI_labels: labels of total proposals
        bbox_targets: target transformations for positive proposals only
        pos_indices: Indices of positive proposals to be used in bbox regression loss
        '''
        boxes = boxes.reshape(-1, 4)
        labels = labels.flatten()
        ROIs, ROI_labels, bbox_targets, pos_indices = sample_proposals(ROIs, boxes, labels)
        ROIs = torch.tensor(np.array(ROIs)).to(DEVICE) # Converting first into np.array then to Tensor for spped (list -> np.array -> tensor)
        ROI_labels = ROI_labels.to(DEVICE)
        bbox_targets = bbox_targets.to(DEVICE)
        pos_indices = torch.tensor(pos_indices).to(DEVICE)

        # Forward pass
        image = image.permute(0, 3, 1, 2)
        # ROIs is of shape [K, 4] (without batch index) and batch size is 1
        batch_indices = torch.zeros((ROIs.size(0), 1), device=ROIs.device)  # Batch index 0 for all ROIs
        ROIs_with_batch_index = torch.cat([batch_indices, ROIs], dim=1)  # Concatenate batch index with ROIs
        # ROIs_with_batch_index will have shape [K, 5]
        cls_scores, bbox_preds = model(image, ROIs_with_batch_index)

        # Compute loss
        '''
        cls_scores : [64, num_classes + 1]
        bbox_preds : [64, num_classes * 4]
        ROI_labels : [64, 1]
        bbox_targets : [len(pos_indices), 4]
        '''
        loss = loss_fn(cls_scores, bbox_preds, ROI_labels, bbox_targets, pos_indices)
        train_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred = torch.argmax(cls_scores, dim=1)
        correct += pred.eq(ROI_labels.view_as(pred)).sum().item()
        total_proposals += 64
    
    train_loss /= len(dataloader)
    train_acc = 100. * correct / total_proposals
    return train_loss, train_acc

In [102]:
import time
import copy

loss_lst, acc_lst = [], []
def fine_tune(model: torch.nn.Module, 
              train_dataloader: torch.utils.data.DataLoader, 
              optimizer: torch.optim.Optimizer,
              loss_fn: torch.nn.Module=MultiTaskLoss(),
              num_epochs: int=10):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
        start_time = time.time()
        # Train the model and print save the results
        train_loss, train_acc = train(model=model,
                                      dataloader=train_dataloader, 
                                      optimizer=optimizer,
                                      loss_fn=loss_fn)
        # Storing loss and acc for the epoch
        loss_lst.append(train_loss)
        acc_lst.append(train_acc)
        
        if train_acc > best_acc:
            best_acc = train_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        end_time = time.time()
        time_elapsed = end_time - start_time
        print(f"------------ epoch {epoch} ------------")
        print(f"Train loss: {train_loss:.4f} | Train acc: {train_acc:.2f}%")
        print(f"Time taken: {time_elapsed / 60:.0f}min {time_elapsed % 60:.0f}s")
        
    model.load_state_dict(best_model_wts)
    return model

In [103]:
optimizer = torch.optim.SGD([
    {'params': fastrcnn.features.parameters(), 'lr':1e-4},
    {'params': fastrcnn.classifier.parameters(), 'lr':1e-3},
    {'params': fastrcnn.softmax.parameters(), 'lr':1e-3},
    {'params': fastrcnn.bbox.parameters(), 'lr':1e-3},
], momentum=0.9, weight_decay=0.0005)

In [104]:
fastrcnn_FineTuned = fine_tune(fastrcnn, train_dataloader, optimizer)

/Users/h383kim/miniforge3/envs/env/lib/python3.8/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


------------ epoch 1 ------------
Train loss: 3.0049 | Train acc: 50.47%
Time taken: 4min 45s


KeyboardInterrupt: 

In [ ]:
# Train Loss Graph
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(loss_lst)
plt.xlabel("Epoch")
plt.ylabel("Train Loss")
plt.title("Train Loss Graph")

# Train Accuracy Graph
plt.subplot(1, 2, 2)
plt.plot(acc_lst)
plt.xlabel("Epoch")
plt.ylabel("Train Accuracy (%)")
plt.title("Train Accuracy Graph")